# WhisperX Cloud API - 极简安装版 🚀

**核心改进**: 使用 Mamba 一键安装所有依赖（包括预编译的 WhisperX）

## 使用步骤
1. 设置 GPU: Runtime → Change runtime type → GPU
2. 设置 ngrok token (在下面填写)
3. Run All
4. 复制生成的 URL 到 VideoLingo

In [ ]:
# ============================================
# 配置区域
# ============================================
NGROK_TOKEN = ""  # <-- 填写你的 ngrok token (从 https://dashboard.ngrok.com/get-started/your-authtoken 获取)

# 可选: 设置 HuggingFace 镜像（中国大陆）
# import os
# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [ ]:
# ============================================
# Step 1: 一键安装 (Mamba 版)
# ============================================
import subprocess
import os
import sys
import json

# 检测环境
IN_COLAB = 'google.colab' in sys.modules
IN_KAGGLE = os.path.exists('/kaggle')

if IN_COLAB:
    ENV_PATH = '/content/conda-envs/whisperx-cloud'
elif IN_KAGGLE:
    ENV_PATH = '/kaggle/working/conda-envs/whisperx-cloud'
else:
    ENV_PATH = os.path.expanduser('~/conda-envs/whisperx-cloud')

print(f"🚀 开始安装 WhisperX Cloud...")
print(f"环境路径: {ENV_PATH}")

# 1. 安装 Mamba (Miniforge)
mamba_bin = os.path.expanduser('~/miniforge3/bin/mamba')
if not os.path.exists(mamba_bin):
    print("\n📦 安装 Mamba...")
    !wget -q -O /tmp/miniforge.sh https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh
    !bash /tmp/miniforge.sh -b -p ~/miniforge3
    print("✅ Mamba 安装完成")
else:
    print("✅ Mamba 已安装")

# 2. 创建环境并安装所有包 (conda-forge 有预编译的 whisperx!)
print("\n📦 创建环境并安装 WhisperX (使用 conda-forge 预编译包)...")
print("⏳ 这可能需要 5-10 分钟，请耐心等待...")

# 清理旧环境
if os.path.exists(ENV_PATH):
    !{mamba_bin} remove -p {ENV_PATH} --all -y -q

# 一键安装所有依赖
!{mamba_bin} create -p {ENV_PATH} \
    -c conda-forge -c pytorch -c nvidia -y \
    python=3.10 \
    pytorch=2.0.0 \
    torchaudio=2.0.0 \
    pytorch-cuda=11.8 \
    ffmpeg \
    whisperx \
    fastapi \
    uvicorn \
    python-multipart \
    pyngrok \
    requests \
    nest-asyncio

print("\n✅ 环境安装完成!")

In [ ]:
# ============================================
# Step 2: 验证安装
# ============================================
python_path = f"{ENV_PATH}/bin/python"

print("🔍 验证关键包...")
for pkg in ['torch', 'whisperx', 'fastapi', 'pyngrok']:
    result = subprocess.run([python_path, '-c', f'import {pkg}; print("OK")'],
                          capture_output=True, text=True)
    status = "✅" if result.returncode == 0 else "❌"
    print(f"  {status} {pkg}")

# 保存配置
config = {'python_path': python_path, 'env_prefix': ENV_PATH}
with open('.conda_python_path', 'w') as f:
    json.dump(config, f)

print(f"\n📝 配置已保存")
print(f"Python: {python_path}")

In [ ]:
# ============================================
# Step 3: 下载服务器代码并启动
# ============================================
import urllib.request

# 下载 whisperx_server.py (如果当前目录没有)
server_url = "https://raw.githubusercontent.com/infinite-gaming-studio/VideoLingo/main/whisperx_cloud/whisperx_server.py"
if not os.path.exists('whisperx_server.py'):
    print("📥 下载服务器代码...")
    urllib.request.urlretrieve(server_url, 'whisperx_server.py')
    print("✅ 下载完成")
else:
    print("✅ 服务器代码已存在")

In [ ]:
# ============================================
# Step 4: 配置 ngrok
# ============================================
if not NGROK_TOKEN:
    raise ValueError("请填写 NGROK_TOKEN! 从 https://dashboard.ngrok.com/get-started/your-authtoken 获取")

# 配置 ngrok
!{python_path} -m pyngrok config add-authtoken {NGROK_TOKEN}
print("✅ ngrok 配置完成")

In [ ]:
# ============================================
# Step 5: 启动服务器
# ============================================
import threading
import time
from pyngrok import ngrok

# 启动 ngrok 隧道
public_url = ngrok.connect(8000, "http")
print(f"🌐 公共 URL: {public_url}")
print(f"\n📋 请将此 URL 复制到 VideoLingo 配置:")
print(f"   whisperX_cloud_url: '{public_url}'")

# 在后台启动服务器
def start_server():
    !{python_path} whisperx_server.py --port 8000

server_thread = threading.Thread(target=start_server)
server_thread.daemon = True
server_thread.start()

print("\n🚀 服务器启动中...")
time.sleep(5)
print("✅ 服务器应该已在运行")

In [ ]:
# ============================================
# Step 6: 测试 API
# ============================================
import requests

try:
    response = requests.get(f"{public_url}/health", timeout=10)
    if response.status_code == 200:
        print("✅ API 测试成功!")
        print(f"响应: {response.json()}")
    else:
        print(f"⚠️ API 返回状态码: {response.status_code}")
except Exception as e:
    print(f"⚠️ 测试失败: {e}")
    print("服务器可能还在启动中，请等待 30 秒后重试")

In [ ]:
# ============================================
# Step 7: 保持运行 (不要停止此单元格!)
# ============================================
print("🟢 服务器运行中...")
print(f"公共 URL: {public_url}")
print("\n⚠️ 不要停止此单元格，否则服务器会关闭!")
print("\n按 Ctrl+C 停止服务器")

try:
    while True:
        time.sleep(60)
        # 保持活跃
except KeyboardInterrupt:
    print("\n🛑 停止服务器...")
    ngrok.disconnect(public_url)
    print("✅ 已关闭")